In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [3]:
# 데이터 로드
article_df = pd.read_csv("article.csv")
len(article_df)

600

In [4]:
# 데이터 전처리 : 데이터 셋 분리
X_train, X_test, y_train, y_test = train_test_split(
article_df.content, article_df.category, test_size=0.1, random_state=1)
len(X_train), len(X_test), len(y_train), len(y_test)

(540, 60, 540, 60)

In [5]:
# 모델 만들기
clf = Pipeline([
    ('vect', TfidfVectorizer()),
    ('clf', MultinomialNB(alpha=0.1)),
])

In [6]:
# 학습
model = clf.fit(X_train.values.astype("str"), y_train)

In [7]:
# 모델 성능 확인
y_pred = model.predict(X_test)

In [10]:
list(y_test[:5]), list(y_pred[:5])

([104, 104, 105, 104, 102], [104, 104, 105, 104, 102])

In [11]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         100       0.60      0.90      0.72        10
         101       0.56      0.62      0.59         8
         102       0.43      0.30      0.35        10
         103       1.00      0.29      0.44         7
         104       1.00      0.81      0.90        16
         105       0.64      1.00      0.78         9

    accuracy                           0.68        60
   macro avg       0.70      0.65      0.63        60
weighted avg       0.73      0.68      0.67        60



#### 모델 사용하기

In [16]:
categories = {
    100: "정치",
    101: "경제",
    102: "사회",
    103: "생활/문화",
    104: "세계",
    105: "IT/과학",
}

In [13]:
contents = [
    "네이버와 카카오는 드론 기술 발전에 주력",
    "요즘 환율 주가 예측이 불가",
    "광화문 대한애국당 천막, 격렬 저항 속 철거",
]

In [15]:
datas = {
    "content": contents,
    "category_code": model.predict(contents),
}
df = pd.DataFrame(datas)
df

,content,category_code
0,네이버와 카카오는 드론 기술 발전에 주력,105
1,요즘 환율 주가 예측이 불가,100
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",103


In [20]:
round(max(model.predict_proba([contents[0]])[0]), 2)

0.5

In [21]:
df["category"] = df["category_code"].apply(lambda data: categories[data])
df["proba"] = df["content"].apply(
    lambda data: round(max(model.predict_proba([data])[0]), 2) )
df

,content,category_code,category,proba
0,네이버와 카카오는 드론 기술 발전에 주력,105,IT/과학,0.50
1,요즘 환율 주가 예측이 불가,100,정치,0.22
2,"광화문 대한애국당 천막, 격렬 저항 속 철거",103,생활/문화,0.17


In [22]:
# 모델 저장

In [23]:
pickle.dump(model, open("clf.pkl", "wb"))